<a href="https://colab.research.google.com/github/nnilayy/Yolact-Edge/blob/main/Yolact_Edge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CLONING THE YOLACT REPO

In [ ]:
%cd /content
!git clone https://github.com/haotian-liu/yolact_edge.git
!git clone https://github.com/chentinghao/download_google_drive.git

%cd yolact_edge
!mkdir -p weights

!python ../download_google_drive/download_gdrive.py 1EAzO-vRDZ2hupUJ4JFSUi40lAZ5Jo-Bp /content/weights/yolact_edge_54_800000.pth

In [ ]:
!python3 -m pip install gitpython

##MOUNTING OUR GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive

##MOVING MODEL WEIGHTS FROM RANDOM DRIVE TO COLAB

In [ ]:
# Create the Folder Structure in Colab to store weights
!mkdir /content/weights

# Provide the model_name, file_id, and colab path to store the model weights
model_name = "yolact_edge_54_800000"
file_id = "1L4N4VcykqE-D5JUgWW9zBd6WKmZPBAZQ"
colab_path = "/content/weights"

# Download the model weights
!gdown "{file_id}&confirm=t" --output='{colab_path}/{model_name}.pth'

##MOVING MODEL WEIGHTS FROM DRIVE TO COLAB

In [ ]:
# Create the Folder Structure in Colab to store weights
!mkdir /content/yolact
!mkdir /content/yolact/model_weights

# Provide the drive path and the colab_path
drive_path = '/content/drive/MyDrive/yolact_plus_resnet50_54_800000.pth'
colab_path = '/content/yolact/model_weights/yolact_plus_resnet50_54_800000.pth'

# Move the weights from drive to colab
!cp "{drive_path}" "{colab_path}"

##CUSTOM DATASET
* Drop the file with your custom dataset in the /content by dragging and dropping as a zip folder.
*Unzip the zip folder as follows:
```
!unzip -q /content/custom_dataset.zip -d /content/
```
*Click on the following config file link: /content/yolact_edge/yolact_edge/data/config.py.
*Under this file, go to "DATASETS" and add the following. 
```
my_custom_dataset = dataset_base.copy({
    'name': 'My Dataset',

    'train_images': 'path_to_training_images',
    'train_info':   'path_to_training_annotation',

    'valid_images': 'path_to_validation_images',
    'valid_info':   'path_to_validation_annotation',

    'has_gt': True,
    'class_names': ('my_class_id_1', 'my_class_id_2', 'my_class_id_3', ...)
})
```
*In the above, add the path to your train and test dataset and annotations files, from the custom dataset you unziped. 
*Then under the YOLACT v1.0 CONFIGS section and change the dataset to custom_dataset



In [ ]:
!unzip -q /content/main_images.zip -d /content/

##TRAINING THE MODEL

* If you encounter the error: *RuntimeError: Expected a ‘cuda’ device type for generator but found ‘cpu’*,
*Click the following link for the test.py file: /content/yolact/train.py, and go to line 251 and change shuffle=True to shuffle=False


* To change the number of iterations, check under CONFIG DEFAULTS, and under that *max_iter*


In [ ]:
!python /content/yolact_edge/train.py --config=yolact_base_config --resume=/content/yolact_edge/weights/yolact_base_99_100.pth

In [ ]:
!python3 /content/yolact_edge/train.py --help


In [ ]:
# Trains using the base config with a batch size of 8 (the default).
# !python /content/yolact/train.py --config=yolact_base_config

# Trains yolact_base_config with a batch_size of 5. For the 550px models, 1 batch takes up around 1.5 gigs of VRAM, so specify accordingly.
# !python /content/yolact/train.py --config=yolact_base_config --batch_size=5

# Resume training yolact_base with a specific weight file and start from the iteration specified in the weight file's name.
# !python /content/yolact/train.py --config=yolact_base_config --resume=weights/yolact_base_10_32100.pth --start_iter=-1

# Use the help option to see a description of all available command line arguments
# !python /content/yolact/train.py --help

In [ ]:
# Pre-train the image based model
# !python train.py --config=yolact_edge_youtubevis_config

# Train the flow (warping) module
# !python train.py --config=yolact_edge_vid_trainflow_config --resume=./weights/yolact_edge_youtubevis_847_50000.pth

# Fine tune the network jointly
# !python train.py --config=yolact_edge_vid_config --resume=./weights/yolact_edge_vid_trainflow_144_100000.pth


##TESTING THE MODEL

In [ ]:
!python /content/yolact_edge/eval.py --trained_model=/content/yolact_edge/weights/yolact_base_99_100.pth --config=yolact_base_config --score_threshold=0.15 --top_k=15 --display --image=/content/images.jpg:/content/output.jpg 

# Display qualitative results on the specified image: --image=my_image.png 
# Process a whole folder of images: --images=path/to/input/folder:path/to/output/folder 
# Process an image and save it to another file: --image=input_image.png:output_image.png 

# Display a video in real-time. "--video_multiframe" will process that many frames at once for improved performance.
# If you want, use "--display_fps" to draw the FPS directly on the frame.
# --video_multiframe=4 --video=my_video.mp4 

# Display a webcam feed in real-time. If you have multiple webcams pass the index of the webcam you want instead of 0.
# --video_multiframe=4 --video=0 

# Process a video and save it to another file. This uses the same pipeline as the ones above now, so it's fast!
# --video_multiframe=4 --video=input_video.mp4:output_video.mp4 


[12/24 17:35:04 yolact.eval]: Loading model...
[12/24 17:35:06 yolact.eval]: Model loaded.


##DISPLAYING THE RESULTS 


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

output_images = Path('output_images')

def show_image(img_path):
  img = cv2.imread(img_path)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(16,16))
  plt.imshow(img_cvt)
  plt.show()

# Iterate through all of the output images and display them
for img_path in output_images.iterdir():
  print(img_path)
  show_image(str(img_path))

##MOVING MODEL WEIGHTS FROM COLAB TO DRIVE

In [ ]:
# Create the Folder Structure in Drive to store weights
!mkdir /content/drive/MyDrive/Yolact
!mkdir /content/drive/MyDrive/Yolact/Balls_Weigths

# Provide the drive path and the colab_path
colab_path = '/content/weights/yolact_base_999_1000.pth'
drive_path = '/content/drive/MyDrive/Yolact/Balls_Weigths/yolact_base_999_1000'

# Move the weights from drive to colab
!cp "{colab_path}" "{drive_path}"

In [ ]:
balls_dataset = dataset_base.copy({
   'name': 'balls_dataset',

   'train_images': '/content/balls/train/images',
   'train_info':   '/content/balls/train/annotations/train.json',

   'valid_images': '/content/balls/val/images',
   'valid_info':   '/content/balls/val/annotations/val.json',

   'has_gt': True,
   'class_names': ('Balls')
})

##CLONING THE YOLACT REPO

In [ ]:
%cd /content
!git clone https://github.com/haotian-liu/yolact_edge.git
!git clone https://github.com/chentinghao/download_google_drive.git

%cd yolact_edge
!mkdir -p weights

!python ../download_google_drive/download_gdrive.py 1EAzO-vRDZ2hupUJ4JFSUi40lAZ5Jo-Bp /content/weights/yolact_edge_54_800000.pth

In [ ]:
!python3 -m pip install gitpython

##MOUNTING OUR GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive

##MOVING MODEL WEIGHTS FROM RANDOM DRIVE TO COLAB

In [ ]:
# Create the Folder Structure in Colab to store weights
!mkdir /content/weights

# Provide the model_name, file_id, and colab path to store the model weights
model_name = "yolact_edge_mobilenetv2_54_800000.pth"
file_id = "1L4N4VcykqE-D5JUgWW9zBd6WKmZPBAZQ"
colab_path = "/content/weights"

# Download the model weights
!gdown "{file_id}&confirm=t" --output='{colab_path}/{model_name}.pth'

mkdir: cannot create directory ‘/content/weights’: File exists
Downloading...
From: https://drive.google.com/uc?id=1L4N4VcykqE-D5JUgWW9zBd6WKmZPBAZQ&confirm=t
To: /content/weights/yolact_edge_mobilenetv2_54_800000.pth.pth
100% 36.3M/36.3M [00:00<00:00, 46.8MB/s]


##MOVING MODEL WEIGHTS FROM DRIVE TO COLAB

In [ ]:
# Create the Folder Structure in Colab to store weights
!mkdir /content/yolact
!mkdir /content/yolact/model_weights

# Provide the drive path and the colab_path
drive_path = '/content/drive/MyDrive/yolact_plus_resnet50_54_800000.pth'
colab_path = '/content/yolact/model_weights/yolact_plus_resnet50_54_800000.pth'

# Move the weights from drive to colab
!cp "{drive_path}" "{colab_path}"

##CUSTOM DATASET
* Drop the file with your custom dataset in the /content by dragging and dropping as a zip folder.
*Unzip the zip folder as follows:
```
!unzip -q /content/custom_dataset.zip -d /content/
```
*Click on the following config file link: /content/yolact_edge/yolact_edge/data/config.py.
*Under this file, go to "DATASETS" and add the following. 
```
my_custom_dataset = dataset_base.copy({
    'name': 'My Dataset',

    'train_images': 'path_to_training_images',
    'train_info':   'path_to_training_annotation',

    'valid_images': 'path_to_validation_images',
    'valid_info':   'path_to_validation_annotation',

    'has_gt': True,
    'class_names': ('my_class_id_1', 'my_class_id_2', 'my_class_id_3', ...)
})
```
*In the above, add the path to your train and test dataset and annotations files, from the custom dataset you unziped. 
*Then under the YOLACT v1.0 CONFIGS section and change the dataset to custom_dataset



In [ ]:
!unzip -q /content/main_images.zip -d /content/

##TRAINING THE MODEL

* If you encounter the error: *RuntimeError: Expected a ‘cuda’ device type for generator but found ‘cpu’*,
*Click the following link for the test.py file: /content/yolact/train.py, and go to line 251 and change shuffle=True to shuffle=False


* To change the number of iterations, check under CONFIG DEFAULTS, and under that *max_iter*


In [ ]:
!python3 /content/yolact_edge/train.py --config=yolact_edge_mobilenetv2_config --resume=/content/weights/yolact_edge_mobilenetv2_54_800000.pth.pth

In [ ]:
!python3 /content/yolact_edge/train.py --help


In [ ]:
# Trains using the base config with a batch size of 8 (the default).
# !python /content/yolact/train.py --config=yolact_base_config

# Trains yolact_base_config with a batch_size of 5. For the 550px models, 1 batch takes up around 1.5 gigs of VRAM, so specify accordingly.
# !python /content/yolact/train.py --config=yolact_base_config --batch_size=5

# Resume training yolact_base with a specific weight file and start from the iteration specified in the weight file's name.
# !python /content/yolact/train.py --config=yolact_base_config --resume=weights/yolact_base_10_32100.pth --start_iter=-1

# Use the help option to see a description of all available command line arguments
# !python /content/yolact/train.py --help

In [ ]:
# Pre-train the image based model
# !python train.py --config=yolact_edge_youtubevis_config

# Train the flow (warping) module
# !python train.py --config=yolact_edge_vid_trainflow_config --resume=./weights/yolact_edge_youtubevis_847_50000.pth

# Fine tune the network jointly
# !python train.py --config=yolact_edge_vid_config --resume=./weights/yolact_edge_vid_trainflow_144_100000.pth


##TESTING THE MODEL

In [ ]:
!python /content/yolact_edge/eval.py --config=yolact_edge_mobilenetv2_config --disable_tensorrt --trained_model=/content/yolact_edge/weights/yolact_edge_mobilenetv2_99_100.pth --score_threshold=0.80 --top_k=15 --image=/content/images.jpg:/content/output.jpg 

# Display qualitative results on the specified image: --image=my_image.png 
# Process a whole folder of images: --images=path/to/input/folder:path/to/output/folder 
# Process an image and save it to another file: --image=input_image.png:output_image.png 

# Display a video in real-time. "--video_multiframe" will process that many frames at once for improved performance.
# If you want, use "--display_fps" to draw the FPS directly on the frame.
# --video_multiframe=4 --video=my_video.mp4 

# Display a webcam feed in real-time. If you have multiple webcams pass the index of the webcam you want instead of 0.
# --video_multiframe=4 --video=0 

# Process a video and save it to another file. This uses the same pipeline as the ones above now, so it's fast!
# --video_multiframe=4 --video=input_video.mp4:output_video.mp4 


[12/24 18:23:07 yolact.eval]: Loading model...
[12/24 18:23:09 yolact.eval]: Model loaded.


##DISPLAYING THE RESULTS 


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

output_images = Path('output_images')

def show_image(img_path):
  img = cv2.imread(img_path)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(16,16))
  plt.imshow(img_cvt)
  plt.show()

# Iterate through all of the output images and display them
for img_path in output_images.iterdir():
  print(img_path)
  show_image(str(img_path))

##MOVING MODEL WEIGHTS FROM COLAB TO DRIVE

In [ ]:
# Create the Folder Structure in Drive to store weights
!mkdir /content/drive/MyDrive/Yolact
!mkdir /content/drive/MyDrive/Yolact/Balls_Weigths

# Provide the drive path and the colab_path
colab_path = '/content/weights/yolact_base_999_1000.pth'
drive_path = '/content/drive/MyDrive/Yolact/Balls_Weigths/yolact_base_999_1000'

# Move the weights from drive to colab
!cp "{colab_path}" "{drive_path}"

In [ ]:
balls_dataset = dataset_base.copy({
   'name': 'balls_dataset',

   'train_images': '/content/balls/train/images',
   'train_info':   '/content/balls/train/annotations/train.json',

   'valid_images': '/content/balls/val/images',
   'valid_info':   '/content/balls/val/annotations/val.json',

   'has_gt': True,
   'class_names': ('Balls')
})